## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [35]:
path = '../03-corpus/2-data/1-fr/'
acteur = "chum"

from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='utf-8') as f:
    csv = read_csv(f, sep=',')[["url", "text"]]

csv

,url,text
0,https:////www.chumontreal.qc.ca//,BIENVENUE SUR LE SITE WEB DU CHUM Pour trouver...
1,https:////www.chumontreal.qc.ca//accueil,BIENVENUE SUR LE SITE WEB DU CHUM Pour trouver...
2,https:////www.chumontreal.qc.ca//user//login,Onglets principaux Se connecter (onglet actif)...
3,https:////www.chumontreal.qc.ca//patients,Patients Patient(e)s ou proches? Une nouvelle ...
4,https:////www.chumontreal.qc.ca//enseignement-...,Enseignement et Académie CHUM JE VEUX FAIRE UN...
...,...,...
2283,https:////www.chumontreal.qc.ca//patients//cic...,Tout savoir sur le traitement par plaque radio...
2284,https:////www.chumontreal.qc.ca//patients//cic...,Qu’est-ce que la stéréotaxie et le CyberKnife?...
2285,https:////www.chumontreal.qc.ca//patients//cic...,Comment se passe l’énucléation de l’oeil suiva...
2286,https:////www.chumontreal.qc.ca//patients//cic...,Une équipe à l’affût des innovations en format...


In [36]:
liste = [lien.replace('//', "/") for lien in csv['url'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien and not 'nouvelles' in lien and not 'pdf' in lien and not 'gaspesie' in lien]
liste 

['https://www.chumontreal.qc.ca/',
 'https://www.chumontreal.qc.ca/accueil',
 'https://www.chumontreal.qc.ca/user/login',
 'https://www.chumontreal.qc.ca/patients',
 'https://www.chumontreal.qc.ca/enseignement-academie',
 'https://www.chumontreal.qc.ca/crchum',
 'https://www.chumontreal.qc.ca/innovation',
 'https://www.chumontreal.qc.ca/carrieres',
 'https://www.chumontreal.qc.ca/patients/cofr',
 'https://www.chumontreal.qc.ca/prendre-rendez-vous',
 'https://www.chumontreal.qc.ca/patients/je-cherche-de-l-information-sur-ma-sante',
 'https://www.chumontreal.qc.ca/patients/je-cherche-de-l-information-sur-une-unite-ou-un-service',
 'https://www.chumontreal.qc.ca/covid-19-information-pour-tous',
 'https://www.chumontreal.qc.ca/patients/urgence',
 'https://www.chumontreal.qc.ca/patients/temoignages',
 'https://www.chumontreal.qc.ca/fiches-sante',
 'https://www.chumontreal.qc.ca/services-de-benevolat-animation-et-loisirs',
 'https://www.chumontreal.qc.ca/labo-chum',
 'https://www.chumontreal

In [37]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens = []
for i in range (2, 9): # On prend les liens de niveau 2 à 8
    try:
        liens += [lien for lien in liste if len(lien.split('/')) > i]
    except Exception as e:
        True

In [38]:
regroupements = {}

for lien in liens:
    for i in range(3,9):
        try:
            tag = lien.split('/')[i-1]
            if tag in regroupements:
                regroupements[tag].append(lien)
                
            else:
                if len(tag)>1:
                    regroupements[tag] = []

        except Exception as e:
            True

tags = list(regroupements.keys())

In [39]:
tags = [tag for tag in regroupements if len(regroupements[tag]) > 40]

In [40]:
tags

['www.chumontreal.qc.ca',
 'patients',
 'enseignement-academie',
 'crchum',
 'je-cherche-de-l-information-sur-une-unite-ou-un-service',
 'temoignages',
 'chercheurs',
 'evenements',
 'taxonomy',
 'term',
 'fiche',
 'delete',
 'edit',
 'translations',
 'add',
 'cicc',
 'cancer-du-sein',
 'plateformes-services',
 'centre-apprentissage-organisation-evenements',
 'centre-cardiovasculaire',
 'types-de-cancer',
 'parcours-locomotive',
 'cancer-oculaire']

In [41]:
print('Ça nous donne {} tags.'.format(len(tags)))
tags

Ça nous donne 23 tags.


['www.chumontreal.qc.ca',
 'patients',
 'enseignement-academie',
 'crchum',
 'je-cherche-de-l-information-sur-une-unite-ou-un-service',
 'temoignages',
 'chercheurs',
 'evenements',
 'taxonomy',
 'term',
 'fiche',
 'delete',
 'edit',
 'translations',
 'add',
 'cicc',
 'cancer-du-sein',
 'plateformes-services',
 'centre-apprentissage-organisation-evenements',
 'centre-cardiovasculaire',
 'types-de-cancer',
 'parcours-locomotive',
 'cancer-oculaire']

In [42]:
import os, shutil
from pathlib import Path
from slugify import slugify

# Créer un sous-dossier où on va stocker des nouveaux fichiers CSV correspondant aux sous-corpus
folder_path ='../03-corpus/2-sous-corpus/'

#Path(folder_path).mkdir(parents=True, exist_ok=True)


for tag in tags:
    file_name =  acteur + "_" + slugify(str(tag).strip('\n'))

    # Aller chercher les documents du sous-corpus dans le CSV du corpus principal et les copier dans un nouveau CSV
    try:
        data = csv[csv["url"].str.match('.*[\-\_\/]'+tag+'.*')]
        data.to_csv(folder_path + file_name + ".csv") #encoding='ISO-8859-1'
    except Exception as e:
        print(tag, str(e))
                    

In [43]:
with open(folder_path + acteur + '_sous-corpus.txt', 'w') as f:
    for file in os.listdir(folder_path):
        f.write(file + '\n')

Après avoir manuellement indexé

In [45]:
from pandas import read_csv
filepath = '../03-corpus/2-sous-corpus/0-semi-automatic_tags_' + acteur + '.csv'
with open(filepath, 'r', encoding = 'utf-8') as f:
    csv = read_csv(f, sep=';')
    
# csv = csv[csv['oui/non'] != 'x']
# csv = csv[csv['oui/non'] != 'X']
csv = csv[csv['Sous-corpus'] != 'x']
csv = csv[csv['Sous-corpus'] != 'X']

csv

,Regroupement dans l'arborescence du site,Sous-corpus
1,chum_cancer-du-sein.csv,Cancer du sein
2,chum_cancer-oculaire.csv,Cancer oculaire
4,chum_centre-cardiovasculaire.csv,Centre cardiovasculaire
5,chum_chercheurs.csv,Chercheurs
6,chum_cicc.csv,CICC
7,chum_crchum.csv,CRCHUM
10,chum_enseignement-academie.csv,Enseignement et académie
13,chum_je-cherche-de-l-information-sur-une-unite...,Services
14,chum_parcours-locomotive.csv,Parcours Locomotive
16,chum_plateformes-services.csv,Services


In [46]:
# tags = list(set([tag for tag in csv['oui/non'].tolist()]))
tags = list(set([tag for tag in csv['Sous-corpus'].tolist()]))

tags

['CICC',
 'Chercheurs',
 'Cancer du sein',
 'Centre cardiovasculaire',
 'Parcours Locomotive',
 'Cancer',
 'Cancer oculaire',
 'Enseignement et académie',
 'CRCHUM',
 'Services']

In [47]:
regroupements = {tag : [] for tag in tags}
for tag in tags:
        tuples = csv[csv['Sous-corpus'] == tag]
        liste = [str(t).strip('\n') for t in tuples["Regroupement dans l'arborescence du site"].tolist()]
        regroupements[tag] = liste


In [48]:
regroupements

{'CICC': ['chum_cicc.csv'],
 'Chercheurs': ['chum_chercheurs.csv'],
 'Cancer du sein': ['chum_cancer-du-sein.csv'],
 'Centre cardiovasculaire': ['chum_centre-cardiovasculaire.csv'],
 'Parcours Locomotive': ['chum_parcours-locomotive.csv'],
 'Cancer': ['chum_types-de-cancer.csv'],
 'Cancer oculaire': ['chum_cancer-oculaire.csv'],
 'Enseignement et académie': ['chum_enseignement-academie.csv'],
 'CRCHUM': ['chum_crchum.csv'],
 'Services': ['chum_je-cherche-de-l-information-sur-une-unite-ou-un-service.csv',
  'chum_plateformes-services.csv']}

In [49]:
import pandas
from slugify import slugify

# Créer de nouveaux CSV contenant les données regroupées selon le tag qui leur est associé
path='../03-corpus/2-sous-corpus/'

for tag in regroupements:

    # Aller chercher les éléments correspondant au sous-corpus dans le csv principal et les copier dans un nouveau CSV 
    dfs = []
    path='../03-corpus/2-sous-corpus/'
    for doc in regroupements[tag]:
        with open(path + doc, 'r', encoding='utf-8') as f:
            csv = read_csv(f)
            dfs.append(csv)

    # Concatenate all data into one DataFrame
    try:
        path='../03-corpus/2-data/1-fr/' + acteur +'/'
        file_path = path + acteur + '_'  + slugify(str(tag)) + '.csv'
        big_frame = pandas.concat(dfs, ignore_index=True)
        big_frame.to_csv(file_path)
        print(tag + "\t" + str(len(big_frame)))

    except Exception as e:
        print(tag, str(e))



CICC	50
Chercheurs	524
Cancer du sein	16
Centre cardiovasculaire	15
Parcours Locomotive	11
Cancer	14
Cancer oculaire	15
Enseignement et académie	20
CRCHUM	905
Services	35
